In [1]:
%%configure
{ "conf": {
            "spark.jars":"s3://rupiahplus-configs/hudi_jar/hudi-spark-bundle.jar,s3://rupiahplus-configs/hudi_jar/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606988695033_0001,pyspark,dead,Link,,


In [6]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, array, ArrayType, DateType, DecimalType
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.appName("Product_Price_Tracking") \
     .config("spark.jars","s3://rupiahplus-configs/hudi_jar/hudi-spark-bundle.jar,s3://rupiahplus-configs/hudi_jar/spark-avro.jar") \
     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
     .config("spark.sql.hive.convertMetastoreParquet", "false") \
     .getOrCreate()
# import org.apache.spark.sql.SaveMode
# import org.apache.spark.sql.functions._
# import org.apache.hudi.DataSourceWriteOptions
# import org.apache.hudi.config.HoodieWriteConfig
# import org.apache.hudi.hive.MultiPartKeysValueExtractor
df=spark.read.format("orc").load('s3://rupiahplus-data-warehouse/stream/banda_etl/t_admin_audit/')
# df.write.format("org.apache.hudi") \
#             .option("hoodie.table.name", "t_admin_audit1") \
#             .option("hoodie.datasource.write.storage.type", "COPY_ON_WRITE") \
#             .option("hoodie.datasource.write.operation", "bulk_insert") \
#             .option("hoodie.datasource.write.recordkey.field","id") \
#             .option("hoodie.datasource.write.precombine.field", "id") \
#             .mode("overwrite") \
#             .save("s3://rupiahplus-data-warehouse/aliyun/banda_real/t_admin_audit1/")
# inputDF = spark.createDataFrame(
#     [
#         ("100", "2015-01-01", "2015-01-01T13:51:39.340396Z"),
#         ("101", "2015-01-01", "2015-01-01T12:14:58.597216Z"),
#         ("102", "2015-01-01", "2015-01-01T13:51:40.417052Z"),
#         ("103", "2015-01-01", "2015-01-01T13:51:40.519832Z"),
#         ("104", "2015-01-02", "2015-01-01T12:15:00.512679Z"),
#         ("105", "2015-01-02", "2015-01-01T13:51:42.248818Z"),
#     ],
#     ["id", "creation_date", "last_update_time"]
# )
# inputDF.show()
# # Specify common DataSourceWriteOptions in the single hudiOptions variable
hudiOptions = {
'hoodie.table.name': 'my_hudi_table',
'hoodie.datasource.write.recordkey.field': 'id',
    #记录键字段。用作HoodieKey中recordKey部分的值。 实际值将通过在字段值上调用.toString()来获得。可以使用点符号指定嵌套字段，例如：a.b.c
'hoodie.datasource.write.partitionpath.field': 'year',
    #分区路径字段。用作HoodieKey中partitionPath部分的值。 通过调用.toString()获得实际的值
'hoodie.datasource.write.precombine.field': 'create_time',
'hoodie.datasource.hive_sync.enable': 'true',
'hoodie.datasource.hive_sync.table': 'my_hudi_table',
'hoodie.datasource.hive_sync.partition_fields': 'etldate',
'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}
# option('hoodie.datasource.write.operation', 'insert')\
#是否为写操作进行插入更新、插入或批量插入。使用bulkinsert将新数据加载到表中，之后使用upsert或insert。 
#批量插入使用基于磁盘的写入路径来扩展以加载大量输入，而无需对其进行缓存。
#如果是大量数据使用bulkinsert
# # Write a DataFrame as a Hudi dataset
df.write\
.format('org.apache.hudi')\
.option('hoodie.datasource.write.operation', 'insert')\
.options(**hudiOptions)\
.mode('overwrite')\
.save('s3://rupiahplus-data-warehouse/aliyun/banda_real_hudi/t_admin_audit1/')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…